<a href="https://colab.research.google.com/github/GuzAnd777/Imersao_Alura_Gemini/blob/main/Imers%C3%A3oAlura_ViagemT%C3%A9cnica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
%pip -q install google-genai

In [250]:
# Configurando a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [251]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [252]:
# Instalar Framework de agentes do Google

!pip install -q google-adk

In [253]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [254]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [255]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [256]:
# --- Constantes ---
CEP_ORIGEM_EMPRESA = "88701-180"
CIDADE_ORIGEM_EMPRESA = "Tubarão, Santa Catarina"

In [257]:
# Agente 1: agente_trajeto (Mantido, já inclui Cidade/Estado de destino)
def agente_trajeto (cep_destino_cliente, data_de_hoje_str):
    agente = Agent(
        name="agente_trajeto",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa de viagens. Sua tarefa é usar a ferramenta de busca (google_search)
        para determinar o trajeto e a logística inicial.

        Diretrizes mandatórias:
        1.  Ponto de partida: {CIDADE_ORIGEM_EMPRESA} (CEP: {CEP_ORIGEM_EMPRESA}).
        2.  Pesquise a **cidade e estado** correspondentes ao CEP de destino: {cep_destino_cliente}.
        3.  Transporte (ida e volta):
            *   Destinos próximos (até 4 horas de carro) no mesmo estado (Santa Catarina): Carro.
            *   Destinos distantes no mesmo estado (Santa Catarina): Ônibus.
            *   Destinos em outro estado: Carro até Aeroporto de Florianópolis (FLN), depois avião para
            aeroporto mais próximo do destino.
        4.  Janela de partida: A partir das 12h de segunda-feira. Última saída de Tubarão: quarta-feira
        às 12h.
            *   Para voos, chegar ao aeroporto de embarque com 3 horas de antecedência.
        5.  Manutenção: Sempre às 8h da manhã no local do cliente.
            *   Se chegar na cidade de destino APÓS as 8h, manutenção no dia seguinte, com hospedagem.
        6.  Retorno: Após as 18h do dia da manutenção, ou dia seguinte se não for possível retornar no mesmo dia.
        7.  Viagem: Ocorrerá uma semana após a data da solicitação ({data_de_hoje_str}), respeitando a
        janela de partida. Calcule as datas exatas de partida e retorno.

        Sua Saída DEVE INCLUIR (formate em Markdown):
        ## Planejamento do Trajeto Inicial
        - **Data da Solicitação:** {data_de_hoje_str}
        - **CEP do Cliente:** {cep_destino_cliente}
        - **Cidade e Estado de Destino:** [Resultado da pesquisa - IMPORTANTE PARA O AGENTE DE SAÚDE]
        ---
        - **DATA DE PARTIDA DA VIAGEM (DE TUBARÃO):** [DD/MM/AAAA]
        - **HORÁRIO DE PARTIDA ESTIMADO (DE TUBARÃO):** [HH:MM]
        - **DATA DE CHEGADA ESTIMADA (NO DESTINO):** [DD/MM/AAAA]
        - **HORÁRIO DE CHEGADA ESTIMADO (NO DESTINO):** [HH:MM]
        ---
        - **DATA DE PARTIDA DO RETORNO (DO DESTINO):** [DD/MM/AAAA]
        - **HORÁRIO DE PARTIDA ESTIMADO (DO DESTINO):** [HH:MM]
        - **DATA DE CHEGADA ESTIMADA (EM TUBARÃO - FINAL):** [DD/MM/AAAA]
        - **HORÁRIO DE CHEGADA ESTIMADO (EM TUBARÃO - FINAL):** [HH:MM]
        ---
        - **Meios de Transporte Utilizados (Ida e Volta):** [Ex: Carro até FLN, depois Avião]
        - **Estimativa de Dias Fora:** [Número de dias]
        - **Necessidade de Hospedagem:** [Sim/Não]
        - **Observações do Trajeto:** [Qualquer informação adicional relevante]
        """,
        description="Agente que busca informações iniciais sobre o trajeto, incluindo cidade/estado de destino.",
        tools=[google_search]
    )
    entrada_agente = f"CEP do cliente: {cep_destino_cliente}\nData da solicitação: {data_de_hoje_str}"
    resultado_trajeto = call_agent(agente, entrada_agente)
    return resultado_trajeto

In [258]:
# Agente 2: Itinerário Detalhado (Foco em opções, não necessariamente o custo final)
def agente_itinerario(cep_destino_cliente, info_trajeto_str):
    agente = Agent(
        name="agente_itinerario",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em montar itinerários de viagem no Brasil.
        Analise CUIDADOSAMENTE as seguintes informações do planejamento de trajeto inicial, que contêm as
        DATAS EXATAS para a viagem:
        --- INÍCIO INFORMAÇÕES DO TRAJETO ---
        {info_trajeto_str}
        --- FIM INFORMAÇÕES DO TRAJETO ---

        Suas tarefas:
        1.  EXTRAIA as datas de partida e retorno do texto acima.
        2.  Se a viagem incluir avião:
            *   Sua prioridade é encontrar opções de voos NAS COMPANHIAS LATAM E AZUL para as DATAS EXTRAÍDAS.
            *   Pesquise usando 'google_search' com termos como `voos [origem] para [destino] Latam [data_ida] [data_volta]
            site:latamairlines.com`.
         3.  Se a viagem incluir ônibus:
            *   Pesquise opções de ônibus para as DATAS EXTRAÍDAS. Mencione companhias.
        4.  Se a viagem incluir carro:
            *   Liste os pedágios e uma estimativa de seus valores.
        5.  Se o técnico for usar Uber na cidade de destino:
            *   Forneça uma estimativa de quantas viagens poderão ser realizadas para os trechos prováveis.

        ---
        **INSTRUÇÕES DE FORMATAÇÃO DA SAÍDA (Markdown):**
        ## 1. Opções de Passagens ([Origem] - [Destino])
        ### Opção Principal Aérea/Ônibus: [Nome da Companhia/Fonte]
        - **Rota:** [Ex: Florianópolis (FLN) para Macapá (MCP)]
        - **Datas Pesquisadas:** IDA: [DATA EXTRAÍDA], VOLTA: [DATA EXTRAÍDA]
        - **Observações:** [Ex: "Este é um preço estimado e sujeito a alterações." ou "Opção encontrada em [site]."]

        ## 2. Pedágios (Trecho: [Ex: Tubarão, SC - Aeroporto de Florianópolis, SC])
        - **Praças de Pedágio Encontradas (Estimativa):** (Liste)
        ---
        Priorize encontrar as opções de transporte.
        """,
        description="Agente que detalha opções de transporte e estimativas.",
        tools=[google_search]
    )
    entrada_agente = f"CEP do cliente: {cep_destino_cliente}\nInformações do Trajeto Base: {info_trajeto_str}"
    resultado_itinerario = call_agent(agente, entrada_agente)
    return resultado_itinerario

In [259]:
# Agente 3: Hospedagem (Foco em encontrar opções, não necessariamente o custo final)
def agente_hospedagem(cep_destino_cliente, info_trajeto_str): # Não precisa mais do info_itinerario_str se o foco não é custo.
    agente = Agent(
        name="agente_hospedagem",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em encontrar sugestões de hospedagem.
        Informações do trajeto: {info_trajeto_str}

        Tarefa:
        1. Verifique no `info_trajeto_str` se a hospedagem é necessária (campo "Necessidade de Hospedagem:").
        2. Se SIM:
            *   Use 'google_search' para encontrar um hotel com boas indicações na cidade de destino
            (identificada no `info_trajeto_str`).
            *   O hotel deve estar a no máximo 2km do local do cliente (CEP: {cep_destino_cliente}).
            *   Indique o nome do hotel e, se encontrar, uma estimativa de diária com a ressalva de que
            precisa ser confirmada.
        3. Se NÃO: Indique que a hospedagem não é necessária.

        ---
        **INSTRUÇÕES DE FORMATAÇÃO DA SAÍDA (Markdown):**
        **Caso a hospedagem SEJA necessária:**
        ## Sugestão de Hospedagem
        - **Necessidade de Hospedagem:** Sim
        - **Hotel Sugerido:** [Nome do Hotel]
        - **Endereço (se encontrado):** [Endereço]
        - **Valor Estimado da Diária (se encontrado):** R$ [Valor (Estimativa, confirmar)]
        - **Observações:** [Ex: "Hotel bem avaliado próximo ao cliente."]

        **Caso a hospedagem NÃO SEJA necessária:**
        ## Sugestão de Hospedagem
        - **Necessidade de Hospedagem:** Não
        - **Hotel Sugerido:** N/A
        ---
        """,
        description="Agente que busca sugestões de hospedagem.",
        tools=[google_search]
    )
    entrada_agente = f"CEP do cliente: {cep_destino_cliente}\nTrajeto: {info_trajeto_str}"
    resultado_hospedagem = call_agent(agente, entrada_agente)
    return resultado_hospedagem

In [260]:
# Agente 4: Alimentação (Mantido como antes)
def agente_alimentacao(info_trajeto_str):
    agente = Agent(
        name="agente_alimentacao",
        model="gemini-2.0-flash",
        instruction=f"""
        Você auxilia o técnico a calcular o número de refeições durante suas viagens.
        O técnico tem direito a uma refeição por período do dia em que está em trânsito ou no
        local de destino: manhã, tarde e noite.
        Com base nas informações do trajeto (principalmente "dias fora" e horários
        de partida/chegada):
        {info_trajeto_str}
        Sua tarefa: Contabilizar o número total de refeições.
        Sua Saída DEVE INCLUIR (Markdown):
        - **Número total de refeições:** [Número]
        """,
        description="Agente que calcula o número total de refeições.",
        tools=[]
    )
    entrada_agente = f"Informações do Trajeto: {info_trajeto_str}"
    resultado_alimentacao = call_agent(agente, entrada_agente)
    return resultado_alimentacao

In [261]:
# Agente 5: Saúde
def agente_saude(info_trajeto_str): # Precisa da cidade/estado do info_trajeto
    agente = Agent(
        name="agente_saude",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de saúde para viajantes.
        Com base nas informações do trajeto, especialmente a **Cidade e Estado de Destino**:
        --- INÍCIO INFORMAÇÕES DO TRAJETO ---
        {info_trajeto_str}
        --- FIM INFORMAÇÕES DO TRAJETO ---

        Sua tarefa é:
        1. Identificar a Cidade e Estado de destino no texto acima.
        2. Usar a ferramenta 'google_search' para pesquisar recomendações de saúde específicas
        para essa localidade no Brasil.
           Busque por:
           - Vacinas recomendadas ou obrigatórias.
           - Riscos de doenças transmitidas por mosquitos (Dengue, Zika, Chikungunya,
           Febre Amarela, Malária).
           - Precauções com água e alimentos.
           - Outras dicas de saúde relevantes para viajantes naquela região.
        3.  Sempre inclua uma recomendação para que o viajante consulte um médico ou um
        Centro de Orientação ao Viajante antes da viagem para aconselhamento personalizado.

        ---
        **INSTRUÇÕES DE FORMATAÇÃO DA SAÍDA (Markdown):**
        ## Recomendações de Saúde para [Cidade Destino], [Estado Destino]

        - **Vacinas Recomendadas/Obrigatórias:**
            - [Vacina 1]: [Breve descrição/recomendação]
            - [Vacina 2]: [Breve descrição/recomendação]
            - (liste todas as relevantes)
        - **Precauções Contra Doenças Transmitidas por Mosquitos:**
            - [Doença 1]: [Precauções específicas ou nível de risco]
            - [Doença 2]: [Precauções específicas ou nível de risco]
            - Dicas gerais: [Ex: Uso de repelente, roupas adequadas]
        - **Cuidados com Água e Alimentos:**
            - [Principais recomendações]
        - **Outras Dicas de Saúde:**
            - [Dicas adicionais]
        ---
        **Atenção:** Estas são recomendações gerais. É fundamental que o técnico **consulte
        um médico ou um Centro de Orientação ao Viajante antes da viagem** para obter
        aconselhamento de saúde personalizado e atualizado para a região de destino.
        ---
        """,
        description="Agente que pesquisa recomendações de saúde para o local de destino.",
        tools=[google_search]
    )
    entrada_agente = f"Informações do Trajeto (para identificar destino): {info_trajeto_str}"
    resultado_saude = call_agent(agente, entrada_agente)
    return resultado_saude

In [262]:
# Agente 6: Revisão (Ajustado para focar na logística e saúde)
def agente_revisao(info_trajeto_str, info_itinerario_str, info_hospedagem_str, info_alimentacao_str, info_saude_str):
    agente = Agent(
        name="agente_revisao",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em conferência de planejamento de viagens.
        Dados para revisão:
        - Trajeto: {info_trajeto_str}
        - Itinerário: {info_itinerario_str}
        - Hospedagem: {info_hospedagem_str}
        - Alimentação: {info_alimentacao_str}
        - Recomendações de Saúde: {info_saude_str}

        Tarefas de Revisão:
        1.  **Consistência Geral:** Verifique se as informações entre as etapas são consistentes
         (ex: necessidade de hospedagem, horários).
        2.  **Aderência às Diretrizes:** Confirme se o planejamento segue as diretrizes da empresa
         (transporte, horários).
        3.  **Recomendações de Saúde:** Avalie se as recomendações de saúde são claras, relevantes
         para o destino (identificado no `info_trajeto_str`) e incluem o aviso para consulta médica.
        4.  **Viabilidade Geral:** O planejamento como um todo parece viável e completo?

        ---
        **INSTRUÇÕES DE FORMATAÇÃO DA SAÍDA (Markdown):**
        ## Relatório de Revisão do Planejamento de Viagem

        ### Status Geral da Revisão:
        - [Ex: "Planejamento Aprovado" ou "Aprovado com Observações" ou "Requer Ajustes"]

        ---
        ### Pontos Verificados e Comentários:
        1.  **Consistência e Lógica do Planejamento (Trajeto, Itinerário, Hospedagem, Alimentação):**
            - **Status:** [Ex: OK / Atenção Necessária]
            - **Comentário:** [Seus comentários sobre a logística]
        2.  **Aderência às Diretrizes da Empresa:**
            - **Status:** [Ex: OK / Não Conforme]
            - **Comentário:** [Seus comentários]
        3.  **Recomendações de Saúde:**
            - **Status:** [Ex: Adequadas / Incompletas / Revisar]
            - **Comentário:** [Ex: "Recomendações de saúde para [Cidade/Estado] parecem completas
            e o alerta para consulta médica está presente."]
        ### Recomendações / Ações Necessárias (se houver):
        - [Listar ações]
        ---
        **Conclusão da Revisão:** [Frase final]
        """,
        description="Agente que revisa o planejamento da viagem com foco na logística e saúde.",
        tools=[] # Principalmente análise dos inputs
    )
    entrada_agente = f"Trajeto: {info_trajeto_str}\nItinerário: {info_itinerario_str}\nHospedagem: {info_hospedagem_str}\nAlimentação: {info_alimentacao_str}\nSaúde: {info_saude_str}"
    resultado_revisao = call_agent(agente, entrada_agente)
    return resultado_revisao

In [263]:
# Agente 7: Relatorio (Ajustado para incluir saúde e remover custos detalhados)
def agente_relatorio(cep_destino_cliente, info_trajeto_str, info_itinerario_str, info_hospedagem_str, info_alimentacao_str, info_saude_str, info_revisao_str):
    agente = Agent(
        name="agente_relatorio_final",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em redigir relatórios concisos de planejamento de viagem.
        Colete todas as informações e redija um documento resumido.

        Dados Completos da Viagem para o CEP {cep_destino_cliente}:
        - Trajeto: {info_trajeto_str}
        - Itinerário (Opções de Transporte): {info_itinerario_str}
        - Hospedagem (Sugestão): {info_hospedagem_str}
        - Alimentação (Nº de Refeições): {info_alimentacao_str}
        - Recomendações de Saúde: {info_saude_str}
        - Comentários da Revisão: {info_revisao_str}

        ---
        **INSTRUÇÕES DE FORMATAÇÃO DA SAÍDA (Markdown):**

        ----------------------------------------------------
        **RELATÓRIO DE PLANEJamento DE VIAGEM TÉCNICA**
        ----------------------------------------------------
        **Data da Solicitação:** [Extrair de info_trajeto_str]
        **CEP Destino:** {cep_destino_cliente}
        **Cidade/Estado Destino:** [Extrair de info_trajeto_str]

        **1. RESUMO DO TRAJETO:**
           - **Dias Fora:** [Extrair de info_trajeto_str]
           - **Meios de Transporte:** [Extrair de info_trajeto_str]
           - **Partida de Tubarão:** [Data e Hora de info_trajeto_str]
           - **Chegada ao Destino:** [Data e Hora de info_trajeto_str]
           - **Partida do Destino (Retorno):** [Data e Hora de info_trajeto_str]
           - **Chegada em Tubarão (Retorno):** [Data e Hora de info_trajeto_str]

        **2. INFORMAÇÕES DE VIAGEM (OPÇÕES E SUGESTÕES):**
           - **Transporte Principal (Aéreo/Ônibus):** [Resumir principal opção do info_itinerario_str,
           ex: "Opção Azul Linhas Aéreas (à confirmar)"]
           - **Transporte Local (Pedágios/Uber):** [Resumir estimativas do info_itinerario_str]
           - **Hospedagem:** [Resumir sugestão do info_hospedagem_str, ex: "Hotel X" ou "Não necessária"]
           - **Alimentação:** [{info_alimentacao_str} refeições previstas]

        **3. RECOMENDAÇÕES DE SAÚDE ([Cidade Destino], [Estado Destino]):**
           - [Sumarizar os pontos principais do {info_saude_str}, como principais vacinas e precauções]
           - **Atenção:** É fundamental que o técnico **consulte um médico** antes da viagem para
           aconselhamento personalizado.

        **4. OBSERVAÇÕES DA REVISÃO:**
           - [Sumarizar o {info_revisao_str}. Se tudo OK, "Planejamento revisado e aprovado." ou similar]
        ----------------------------------------------------
        """,
        description="Agente que gera o relatório final consolidado da viagem com foco em saúde.",
        tools=[]
    )
    entrada_agente = (f"Dados para relatório:\nTrajeto:{info_trajeto_str}\nItinerário:{info_itinerario_str}\n"
                      f"Hospedagem:{info_hospedagem_str}\nAlimentação:{info_alimentacao_str}\n"
                      f"Saúde:{info_saude_str}\nRevisão:{info_revisao_str}")
    resultado_final = call_agent(agente, entrada_agente)
    return resultado_final

In [265]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Validação de Viagem Técnica 🚀")
print("Data da solicitação: ", data_de_hoje)

cep_destino_cliente = input("❓ Por favor, digite o CEP da empresa que receberá a manutenção: ")
print("--------------------------------------------------------------")

# Agente 1
info_trajeto = agente_trajeto(cep_destino_cliente, data_de_hoje)
print("\n--- 📝 Resultado do Agente 1 (Trajeto) ---")
display(to_markdown(info_trajeto))
print("--------------------------------------------------------------")

# Agente 2
info_itinerario = agente_itinerario(cep_destino_cliente, info_trajeto)
print("\n--- 📝 Resultado do Agente 2 (Itinerário Detalhado) ---")
display(to_markdown(info_itinerario))
print("--------------------------------------------------------------")

# Agente 3
info_hospedagem = agente_hospedagem(cep_destino_cliente, info_trajeto)
print("\n--- 📝 Resultado do Agente 3 (Hospedagem) ---")
display(to_markdown(info_hospedagem))
print("--------------------------------------------------------------")

# Agente 4
info_alimentacao = agente_alimentacao(info_trajeto)
print("\n--- 📝 Resultado do Agente 4 (Alimentação) ---")
display(to_markdown(info_alimentacao))
print("--------------------------------------------------------------")

# Agente 5
info_saude = agente_saude(info_trajeto)
print("\n--- 📝 Resultado do Agente 5 (Saúde) ---")
display(to_markdown(info_saude))
print("--------------------------------------------------------------")

# Agente 6
info_revisao = agente_revisao(info_trajeto, info_itinerario, info_hospedagem, info_alimentacao, info_saude)
print("\n--- 📝 Resultado do Agente 6 (Revisão Final) ---")
display(to_markdown(info_revisao))
print("--------------------------------------------------------------")

# Agente 7
relatorio_final = agente_relatorio(cep_destino_cliente, info_trajeto, info_itinerario, info_hospedagem, info_alimentacao, info_custos, info_revisao)
print("\n--- 📑 Resultado do Agente 7 (Relatório Final) ---")
display(to_markdown(relatorio_final))
print("--------------------------------------------------------------")

print("\n🎉 Processo de planejamento de viagem concluído! 🎉")

🚀 Iniciando o Sistema de Validação de Viagem Técnica 🚀
Data da solicitação:  17/05/2025
❓ Por favor, digite o CEP da empresa que receberá a manutenção: 68901-971
--------------------------------------------------------------

--- 📝 Resultado do Agente 1 (Trajeto) ---


> ## Planejamento do Trajeto Inicial
> - **Data da Solicitação:** 17/05/2025
> - **CEP do Cliente:** 68901-971
> - **Cidade e Estado de Destino:** Macapá, Amapá
> ---
> - **DATA DE PARTIDA DA VIAGEM (DE TUBARÃO):** 26/05/2025
> - **HORÁRIO DE PARTIDA ESTIMADO (DE TUBARÃO):** 09:00
> - **DATA DE CHEGADA ESTIMADA (NO DESTINO):** 26/05/2025
> - **HORÁRIO DE CHEGADA ESTIMADO (NO DESTINO):** 23:00
> ---
> - **DATA DE PARTIDA DO RETORNO (DO DESTINO):** 27/05/2025
> - **HORÁRIO DE PARTIDA ESTIMADO (DO DESTINO):** 19:00
> - **DATA DE CHEGADA ESTIMADA (EM TUBARÃO - FINAL):** 28/05/2025
> - **HORÁRIO DE CHEGADA ESTIMADO (EM TUBARÃO - FINAL):** 12:00
> ---
> - **Meios de Transporte Utilizados (Ida e Volta):** Carro até Aeroporto de Florianópolis (FLN), depois Avião para Aeroporto Internacional de Macapá - Alberto Alcolumbre (MCP).
> - **Estimativa de Dias Fora:** 3
> - **Necessidade de Hospedagem:** Sim
> - **Observações do Trajeto:** Partida de Tubarão (SC) na segunda-feira (26/05/2025) às 09:00, chegando em Macapá (AP) no mesmo dia às 23:00. Manutenção agendada para terça-feira (27/05/2025) às 8h, com retorno previsto para Tubarão (SC) após as 19:00, chegando na quarta-feira (28/05/2025) ao meio dia. Será necessário pernoitar em Macapá devido ao horário de chegada.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Itinerário Detalhado) ---


> Compreendido! Analisarei as informações do trajeto fornecidas para detalhar as opções de passagens aéreas, custos de pedágio no trecho de carro e estimativas de uso do Uber em Macapá.
> 
> 
> ## 1. Opções de Passagens (Florianópolis - Macapá)
> 
> ### Opção Principal Aérea: Azul Linhas Aéreas
> - **Rota:** Florianópolis (FLN) para Macapá (MCP)
> - **Datas Pesquisadas:** IDA: 26/05/2025, VOLTA: 27/05/2025
> - **Observações:** A Azul Linhas Aéreas oferece voos de Florianópolis para Macapá. Os preços podem variar dependendo da data e da disponibilidade. Consulte o site da Azul para verificar os preços e horários exatos para as datas da viagem.
> 
> ### Opção Aérea Alternativa: LATAM
> - **Rota:** Florianópolis (FLN) para Macapá (MCP)
> - **Datas Pesquisadas:** IDA: 26/05/2025, VOLTA: 27/05/2025
> - **Observações:** A LATAM também opera voos de Florianópolis para Macapá. É recomendado verificar os preços e a disponibilidade diretamente no site da LATAM.
> 
> ## 2. Pedágios (Trecho: Tubarão, SC - Aeroporto de Florianópolis, SC)
> - **Praças de Pedágio Encontradas (Estimativa):**
>     - BR-101, Km 243, Palhoça.
>     - BR-101, Km 299, Laguna.
>     - BR-101, Km 344, Tubarão.
>     - BR-101, Km 404, Araranguá.
>     - BR-101, Km 457, São João do Sul.
> - **Estimativa de Custo Total:** O valor da tarifa para automóveis em cada praça é de R$ 2,40. Portanto, o custo total estimado de pedágios no trecho Tubarão - Florianópolis é de aproximadamente R$12,00.
> 
> ## 3. Estimativa de uso do Uber em Macapá
> - **Estimativa de Corridas:** É possível realizar diversas corridas de Uber em Macapá durante a estadia, considerando a necessidade de deslocamento do aeroporto para o hotel e do hotel para o local de manutenção, além de outras possíveis locomoções.
> - **Custo por Km:** O custo médio por km rodado no Uber em Macapá é de R$ 1,22.
> - **Exemplos de rotas e preços:**
>     - Aeroporto Internacional de Macapá - Alberto Alcolumbre para Av. Fab, 70: Preço estimado.
>     - Aeroporto Internacional de Macapá - Alberto Alcolumbre para R. Tiradentes, 1104: Preço estimado.
>     - Macapá para Santana: R$166-232.
>     - Conjunto Cidade Macapaba para Pacoval: R$12-16.
> 
> Lembre-se que os valores das passagens aéreas e as estimativas de preços do Uber estão sujeitos a alterações.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Hospedagem) ---


...

--------------------------------------------------------------

--- 📝 Resultado do Agente 6 (Revisão Final) ---


> ## Relatório de Revisão do Planejamento de Viagem
> 
> ### Status Geral da Revisão:
> - [Planejamento Aprovado com Observações]
> 
> ---
> ### Pontos Verificados e Comentários:
> 1.  **Consistência e Lógica do Planejamento (Trajeto, Itinerário, Hospedagem, Alimentação):**
>     - **Status:** [OK]
>     - **Comentário:** O planejamento do trajeto, itinerário, hospedagem e alimentação apresenta uma boa lógica e consistência interna. Os horários de voo e deslocamento parecem razoáveis, e a estimativa de refeições é adequada para o período da viagem.
> 
> 2.  **Aderência às Diretrizes da Empresa:**
>     - **Status:** [OK]
>     - **Comentário:** O planejamento parece aderir às diretrizes da empresa, utilizando transporte aéreo e terrestre conforme necessário.
> 
> 3.  **Recomendações de Saúde:**
>     - **Status:** [Adequadas]
>     - **Comentário:** Recomendações de saúde para Macapá, Amapá parecem completas e o alerta para consulta médica está presente.
> ### Recomendações / Ações Necessárias (se houver):
> - Incluir o custo estimado do estacionamento do carro no aeroporto de Florianópolis.
> - Verificar se o hotel sugerido oferece transfer do aeroporto.
> ---
> **Conclusão da Revisão:** O planejamento está bem estruturado e completo, necessitando apenas de alguns pequenos ajustes para otimizar a experiência do técnico.
> 


--------------------------------------------------------------

--- 📑 Resultado do Agente 7 (Relatório Final) ---


> ```markdown
> ----------------------------------------------------
> **RELATÓRIO DE PLANEJamento DE VIAGEM TÉCNICA**
> ----------------------------------------------------
> **Data da Solicitação:** 17/05/2025
> **CEP Destino:** 68901-971
> **Cidade/Estado Destino:** Macapá, Amapá
> 
> **1. RESUMO DO TRAJETO:**
>    - **Dias Fora:** 3
>    - **Meios de Transporte:** Carro até Aeroporto de Florianópolis (FLN), depois Avião para Aeroporto Internacional de Macapá - Alberto Alcolumbre (MCP).
>    - **Partida de Tubarão:** 26/05/2025 às 09:00
>    - **Chegada ao Destino:** 26/05/2025 às 23:00
>    - **Partida do Destino (Retorno):** 27/05/2025 às 19:00
>    - **Chegada em Tubarão (Retorno):** 28/05/2025 às 12:00
> 
> **2. INFORMAÇÕES DE VIAGEM (OPÇÕES E SUGESTÕES):**
>    - **Transporte Principal (Aéreo/Ônibus):** Opção Azul Linhas Aéreas (à confirmar)
>    - **Transporte Local (Pedágios/Uber):** Estimativa de R$12,00 em pedágios no trajeto Tubarão - Florianópolis. Custo médio do Uber em Macapá de R$ 1,22/km.
>    - **Hospedagem:** Hotel Do Forte, próximo à Fortaleza de São José de Macapá.
>    - **Alimentação:** 8 refeições previstas
> 
> **3. RECOMENDAÇÕES DE SAÚDE (Macapá, Amapá):**
>    - Recomenda-se verificar a necessidade de vacinas como Febre Amarela e profilaxia contra Malária. É importante também o uso de repelentes e roupas que cubram a maior parte do corpo para evitar picadas de mosquitos.
>    - **Atenção:** É fundamental que o técnico **consulte um médico** antes da viagem para
>    aconselhamento personalizado.
> 
> **4. OBSERVAÇÕES DA REVISÃO:**
>    - Planejamento aprovado com observações: Incluir o custo estimado do estacionamento do carro no aeroporto de Florianópolis e verificar se o hotel sugerido oferece transfer do aeroporto.
> ----------------------------------------------------
> ```


--------------------------------------------------------------

🎉 Processo de planejamento de viagem concluído! 🎉
